# Limpieza de datos

*La realidad es que gran parte de su tiempo como científico de datos se dedicará a preparar y limpiar sus datos. Valores atípicos, datos faltantes, datos maliciosos, datos erróneos, datos irrelevantes, datos incoherentes, formato*
*Basura dentro, basura fuera. ¡Mira tus datos! ¡Examínalo!. Y siempre haz esto, no solo cuando no obtengas un resultado que te guste.*

**Analicemos algunos datos de registro web. Todo lo que quiero son las páginas más populares en mi sitio web de noticias sin fines de lucro. ¿Qué tan difícil puede ser eso?**

*¡Tomemos un registro de acceso web y descubramos las páginas más vistas en un sitio web desde él! Suena fácil, ¿verdad?
Configuremos un regex que nos permita analizar una línea de registro de acceso de Apache:*

In [2]:
import re

format_pat= re.compile(
    r"(?P<host>[\d\.]+)\s"
    r"(?P<identity>\S*)\s"
    r"(?P<user>\S*)\s"
    r"\[(?P<time>.*?)\]\s"
    r'"(?P<request>.*?)"\s'
    r"(?P<status>\d+)\s"
    r"(?P<bytes>\S*)\s"
    r'"(?P<referer>.*?)"\s'
    r'"(?P<user_agent>.*?)"\s*'
)

logPath = r"C:\Users\Dell Inspiron\Python_notebook\csv\DataScience-Python3\access_log.txt"

*Ahora vamos a crear un pequeño script para extraer la URL en cada acceso, y vamos a utilizar un diccionario para contar el número de veces que aparece cada uno. Luego lo ordenaremos e imprimiremos las 20 páginas principales. ¿Qué podría salir mal?*

In [5]:
URLCounts = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match= format_pat.match(line)
        if match:
            access = match.groupdict()
            agent = access['user_agent']
            if (not('bot' in agent or 'spider' in agent or 
                    'Bot' in agent or 'Spider' in agent or
                    'W3 Total Cache' in agent or agent =='-')):
                request = access['request']
                fields = request.split()
                if (len(fields) == 3):
                    (action, URL, protocol) = fields
                    if (URL.endswith("/")):
                        if (action == 'GET'):
                            if URL in URLCounts:
                                URLCounts[URL] = URLCounts[URL] + 1
                            else:
                                URLCounts[URL] = 1

results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))

/: 77
/orlando-headlines/: 36
/comics-2/: 12
/world/: 12
/weather/: 4
/australia/: 4
/about/: 4
/national-headlines/: 3
/feed/: 2
/sample-page/feed/: 2
/science/: 2
/technology/: 2
/entertainment/: 1
/san-jose-headlines/: 1
/business/: 1
/travel/feed/: 1


*¡Esto está empezando a parecer más creíble! Pero si profundizaras aún más, encontrarías que las páginas /feed/ son sospechosas, y algunos robots todavía se están deslizando. Sin embargo, es correcto decir que las noticias de Orlando, las noticias mundiales y los cómics son las páginas más populares a las que accede un humano real en este día.*

*La moraleja de la historia es: ¡conozca sus datos! Y siempre cuestione y examine sus resultados antes de tomar decisiones basadas en ellos. Si su empresa toma una mala decisión porque proporcionó un análisis de datos de origen incorrectos, podría meterse en problemas reales. Asegúrese de que las decisiones que tome al limpiar sus datos también sean justificables: ¡no elimine los datos solo porque no respaldan los resultados que desea!*

# La importancia de la normalización de datos. 

*Si su modelo se basa en varios atributos numéricos, ¿son comparables?. Ejemplo: las edades pueden variar de 0 a 100 y los ingresos de 0 mil millones. Algunos modelos pueden no funcionar bien cuando diferentes atributos están en escalas muy diferentes. Puede hacer que algunos atributos cuenten más que otros. El sesgo en los atributos también puede ser un problema*

**Ejemplos**

*La implementación de PCA de scikit-learn tiene una opción de "blanquear" que hace esto por usted. Úsalo. Scikit-Learn tiene un módulo de preprocesamiento con prácticas funciones de normalización y escala. Sus datos pueden tener "sí" y "no" que deben convertirse a "1" o "0"*
*Lea los documentos. La mayoría de las técnicas de minería de datos y aprendizaje automático funcionan bien con datos sin procesar y no normalizados. Pero verifique el que está usando antes de comenzar. ¡No olvides volver a escalar tus resultados cuando hayas terminado!*

**Tratar con valores atípicos.**

*A veces es apropiado eliminar valores atípicos de sus datos de entrenamiento. ¡Haz esto responsablemente! Entiende por qué estás haciendo esto. Por ejemplo: en el filtrado colaborativo, un solo usuario que califique miles de películas podría tener un gran efecto en las calificaciones de todos los demás. Eso puede no ser deseable. Otro ejemplo: en los datos de registro wen, los valores atípicos pueden representar bots u otros agentes que deben descartarse. Pero si alguien realmente quiere el ingreso medio de los ciudadanos estadounidenses por ejemplo.No echas a los multimillonarios solo porque quieres*

*Nuestro viejo amigo Standart Desviation proporcionó una forma basada en principios para clasificar los valores atípicos. Encuentra puntos de datos más que algún múltiplo de una desviación estándar en sus datos de entrenamiento. ¿Qué múltiple? Solo tienes que usar el sentido común*